In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import mne
from pqdm.processes import pqdm

import stat_func
import utilities

mne.set_log_level(verbose='WARNING')

N_JOBS = 1
# N_JOBS = 32

%matplotlib qt

## Let's look at one file

In [2]:
subject = "04"
meg_subject_dir = Path(f'lb_analysis_sparse/import_metadata/sub-{subject}/meg/')
epochs_fname = meg_subject_dir / f"sub-{subject}_task-sparse_metadata_proc-clean_epo.fif"
epochs = mne.read_epochs(epochs_fname)
epochs

Number of events,4971
Events,Node1/Tone4/Comm1/Transition2: 279Node1/Tone4/Comm1/Transition4: 14Node1/Tone4/Comm1/Transition5: 96Node1/Tone4/Comm1/Transition7: 14Node10/Tone7/Comm2/Transition1: 275Node10/Tone7/Comm2/Transition2: 96Node10/Tone7/Comm2/Transition4: 17Node10/Tone7/Comm2/Transition7: 9Node11/Tone8/Comm2/Transition1: 193Node11/Tone8/Comm2/Transition2: 176Node11/Tone8/Comm2/Transition3: 12Node11/Tone8/Comm2/Transition7: 11Node12/Tone3/Comm2/Transition2: 269Node12/Tone3/Comm2/Transition4: 11Node12/Tone3/Comm2/Transition5: 95Node12/Tone3/Comm2/Transition7: 13Node2/Tone9/Comm1/Transition1: 306Node2/Tone9/Comm1/Transition2: 92Node2/Tone9/Comm1/Transition4: 12Node2/Tone9/Comm1/Transition7: 12Node3/Tone2/Comm1/Transition1: 216Node3/Tone2/Comm1/Transition2: 215Node3/Tone2/Comm1/Transition3: 17Node3/Tone2/Comm1/Transition6: 15Node4/Tone6/Comm1/Transition1: 316Node4/Tone6/Comm1/Transition2: 91Node4/Tone6/Comm1/Transition4: 14Node4/Tone6/Comm1/Transition6: 12Node5/Tone5/Comm1/Transition1: 194Node5/Tone5/Comm1/Transition2: 189Node5/Tone5/Comm1/Transition3: 19Node5/Tone5/Comm1/Transition7: 14Node6/Tone11/Comm1/Transition2: 312Node6/Tone11/Comm1/Transition4: 15Node6/Tone11/Comm1/Transition5: 88Node6/Tone11/Comm1/Transition7: 9Node7/Tone10/Comm2/Transition1: 1Node7/Tone10/Comm2/Transition2: 290Node7/Tone10/Comm2/Transition4: 15Node7/Tone10/Comm2/Transition5: 99Node7/Tone10/Comm2/Transition7: 10Node8/Tone12/Comm2/Transition1: 288Node8/Tone12/Comm2/Transition2: 95Node8/Tone12/Comm2/Transition4: 12Node8/Tone12/Comm2/Transition6: 14Node9/Tone1/Comm2/Transition1: 190Node9/Tone1/Comm2/Transition2: 185Node9/Tone1/Comm2/Transition3: 15Node9/Tone1/Comm2/Transition6: 19
Time range,-0.096 – 0.248 sec
Baseline,-0.096 – 0.000 sec


In [3]:
epochs.event_id

{'Node1/Tone4/Comm1/Transition2': 1,
 'Node1/Tone4/Comm1/Transition5': 3,
 'Node10/Tone7/Comm2/Transition1': 5,
 'Node10/Tone7/Comm2/Transition2': 6,
 'Node11/Tone8/Comm2/Transition1': 9,
 'Node11/Tone8/Comm2/Transition2': 10,
 'Node12/Tone3/Comm2/Transition2': 13,
 'Node12/Tone3/Comm2/Transition5': 15,
 'Node2/Tone9/Comm1/Transition1': 17,
 'Node2/Tone9/Comm1/Transition2': 18,
 'Node3/Tone2/Comm1/Transition1': 21,
 'Node3/Tone2/Comm1/Transition2': 22,
 'Node4/Tone6/Comm1/Transition1': 25,
 'Node4/Tone6/Comm1/Transition2': 26,
 'Node5/Tone5/Comm1/Transition1': 29,
 'Node5/Tone5/Comm1/Transition2': 30,
 'Node6/Tone11/Comm1/Transition2': 33,
 'Node6/Tone11/Comm1/Transition5': 35,
 'Node7/Tone10/Comm2/Transition2': 38,
 'Node7/Tone10/Comm2/Transition5': 40,
 'Node8/Tone12/Comm2/Transition1': 42,
 'Node8/Tone12/Comm2/Transition2': 43,
 'Node9/Tone1/Comm2/Transition1': 46,
 'Node9/Tone1/Comm2/Transition2': 47,
 'Node1/Tone4/Comm1/Transition4': 2,
 'Node1/Tone4/Comm1/Transition7': 4,
 'Node1

In [4]:
epochs.average().plot_joint();

qt.qpa.drawing: Layer-backing can not be explicitly controlled on 10.14 when built against the 10.14 SDK


In [5]:
epochs.metadata

,event_name,Node,Tone,Comm,Transition,numTrial,numBlock,numTrialinBlock,Cond
0,Node10/Tone7/Comm2/Transition1,10,7,2,1,1,0,1,1
1,Node11/Tone8/Comm2/Transition1,11,8,2,1,2,0,2,1
2,Node12/Tone3/Comm2/Transition2,12,3,2,2,3,0,3,1
3,Node11/Tone8/Comm2/Transition2,11,8,2,2,4,0,4,1
5,Node9/Tone1/Comm2/Transition2,9,1,2,2,6,0,6,1
...,...,...,...,...,...,...,...,...,...
6715,Node5/Tone5/Comm1/Transition1,5,5,1,1,6716,6,956,1
6716,Node6/Tone11/Comm1/Transition2,6,11,1,2,6717,6,957,1
6717,Node5/Tone5/Comm1/Transition2,5,5,1,2,6718,6,958,1
6718,Node3/Tone2/Comm1/Transition3,3,2,1,3,6719,6,959,2


In [6]:
epochs.metadata['Cond'].unique()

array([1, 3, 2, 4])


## Compute and save contrast

In [7]:
def read_cleaned_epochs(subject):
    # Import epochs
    meg_subject_dir = Path(f'lb_analysis_sparse/import_metadata/sub-{subject}/meg/')
    filename = f'sub-{subject}_task-sparse_metadata_proc-clean_epo.fif'
    epochs_path = meg_subject_dir / filename

    epochs = mne.read_epochs(epochs_path)
    return epochs


def compute_contrast(subject, contrast=('Cond == 1', 'Cond == 2')):
    # Read epochs
    epochs = read_cleaned_epochs(subject)

    # Compute evokeds
    evokeds = [
        epochs[contrast[0]].average(),
        epochs[contrast[1]].average()
    ]

    # Contrast the evokeds
    res = mne.combine_evoked(evokeds, weights=[1, -1])

    # Write contrasted epochs to disk
    outputdir = Path(f'lb_analysis_sparse/ERP_EarlyContrast/sub-{subject}/meg/')
    filename = f'sub-{subject}_task-sparse_metadata_proc-clean_epo.fif'
    # Ensure output directory esists
    outputdir.mkdir(parents=True, exist_ok=True)

    contrast_name = f'{contrast[0]}VS{contrast[1]}'
    contrast_name = contrast_name.replace(' ', '').replace('=', '')

    res.save(outputdir / (f'{filename}_contr-{contrast_name}_ave.fif'), overwrite=True)
    return res

subjects = [
    '04', '06',#'07','08','09','13','14','15','16','18','19',
    # '20','21','22','23','24','25','26','27','28','29','30','32'
] # List of subject to open
contrasts_all_subjects= pqdm(subjects, compute_contrast, n_jobs=N_JOBS)

print(f'Calculated {len(contrasts_all_subjects)} contrasts!')

  0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/2k/nkw3dkx97ssdv08xmdgfnjkc0000gn/T/ipykernel_49817/3637399739.py:33: RuntimeWarning: converting "nave" to integer before saving evoked; this can have a minor effect on the scale of source estimates that are computed using "nave".
  res.save(outputdir / (f'{filename}_contr-{contrast_name}_ave.fif'), overwrite=True)


Calculated 2 contrasts!


/var/folders/2k/nkw3dkx97ssdv08xmdgfnjkc0000gn/T/ipykernel_49817/3637399739.py:33: RuntimeWarning: converting "nave" to integer before saving evoked; this can have a minor effect on the scale of source estimates that are computed using "nave".
  res.save(outputdir / (f'{filename}_contr-{contrast_name}_ave.fif'), overwrite=True)


In [8]:
contrast_grand_average = mne.grand_average(contrasts_all_subjects)
contrast_grand_average.plot_joint();

## Time-by-time decoding in sensor space

In [32]:
# We will train the classifier on all left visual vs auditory trials on MEG
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from mne.decoding import SlidingEstimator, cross_val_multiscore, Scaler, Vectorizer



# Prepare the data
epochs = read_cleaned_epochs('04')
contrast = ('Cond == 2', 'Cond == 3')
metadata_query = f'{contrast[0]} or {contrast[1]}'
epochs_to_contrast = epochs[metadata_query].copy()

X = epochs_to_contrast.get_data()
y = epochs_to_contrast.metadata['Cond'].values


classification_pipeline = make_pipeline(
    StandardScaler(),
    # Scaler(epochs_to_contrast.info),
    # Vectorizer(),
    LogisticRegression(
        solver='lbfgs',
        max_iter=500  # doesn't converge with the default of 100
    )
)

time_decoding_estimator = SlidingEstimator(
    classification_pipeline,
    n_jobs=1,
    scoring='roc_auc'
)

# Now, actually run the classification, and evaluate it via a cross-validation
# procedure.
scores = cross_val_multiscore(
    time_decoding_estimator,
    X,
    y,
    cv=5,
    n_jobs=1
)

# Mean scores across cross-validation splits
scores_mean = np.mean(scores, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epochs.times, scores_mean, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')

Text(0.5, 1.0, 'Sensor space decoding')

In [33]:
epochs = read_cleaned_epochs('06')

print(len(epochs['Cond == 1']), len(epochs['Cond == 2']))

4085 141


In [ ]:
# threshold = None
# cluster_stats, data_array_ch_type, ch_type, times = stat_func.run_cluster_permutation_test_1samp(
#     evokeds_contrast, tmin=0, tmax=0.25, ch_type='mag', nperm=2 ** 12,
#     threshold=threshold, n_jobs=N_JOBS, tail=0
# )


In [ ]:
# compute adjacency
ch_type = 'mag'
adjacency, ch_names = mne.channels.find_ch_adjacency(evokeds_contrast[0].info, ch_type)
tmin, tmax = 0, None

# subset of the data, as array
data = np.array(
    [
        np.transpose(
            evoked.copy().pick(ch_type).crop(tmin, None).data
        )
        for evoked in evokeds_contrast
    ]
)
times = evokeds_contrast[0].copy().crop(tmin, tmax).times
info = evokeds_contrast[0].copy().pick(ch_type).info
data.shape, adjacency.shape, times.shape

((23, 32, 102), (102, 102), (32,))

In [ ]:
from mne.stats import permutation_cluster_1samp_test, spatio_temporal_cluster_1samp_test

n_permutations = 2**12
tail = 0
threshold = None

# stat func
# cluster_stats = permutation_cluster_1samp_test(data_array_chtype, threshold=threshold, n_jobs=n_jobs, verbose=True,
#                                                   tail=tail, n_permutations=2**12, adjacency=adjacency,
#                                                   out_type='indices')

cluster_stats = spatio_temporal_cluster_1samp_test(
    data,
    threshold=threshold,
    n_jobs=N_JOBS,
    tail=tail,
    n_permutations=n_permutations,
    adjacency=adjacency,
    verbose=True,
)

T_obs, clusters, p_values, _ = cluster_stats

Using a threshold of 2.073873
stat_fun(H1): min=-3.585704 max=4.689721
Running initial clustering
Found 27 clusters
Permuting 4095 times...


100%|██████████|  : 4095/4095 [00:24<00:00,  166.45it/s]

Computing cluster p-values
Done.


In [ ]:
p_threshold = 0.2
good_cluster_inds = np.where(p_values < p_threshold)[0]
good_cluster_inds

array([5])

In [ ]:
plt.plot(times, T_obs);

In [ ]:

cluster_info = stat_func.extract_info_cluster(
    cluster_stats, p_threshold, times,
    info, data, ch_type)

We found 102 positions for ch_type mag


In [ ]:

stat_func.plot_clusters(
    cluster_info, ch_type, T_obs_max=3., fname='', figname_initial='',
    filter_smooth=False, outfile=None)

Saving _clust_1.svg


True